## Merging multiple .tex file
## steps :- 
### 1. find root file (main.tex) file 
### 2. use pattern begin{document} for finding root
### 3. After finding root read root file and find pattern for input{file_name.tex} etc.
### 4. substitute the pattern with corresponding .tex file


In [2]:
import os
import re
import subprocess


In [3]:
pattern  = r'(\\input\s*{?([A-Za-z0-9_-]+))|(\\include\s*{?([A-Za-z0-9_-]+))'
pattern2 = r"(\\begin\s*{?document)|(\\begin\s*{?Document)|(\\begin\s*{?DOCUMENT)"
# pattern3_old = r'(%+\s*\\input\s*{?([A-Za-z0-9_\/-]+))|(%+\s*\\include\s*{?([A-Za-z0-9\/_-]+))|(\\input\s*{?([A-Za-z0-9_\/-]+))|(\\include\s*{?([A-Za-z0-9\/_-]+))'
pattern3 = r'(%+\s*\\input\s*{?([A-Za-z0-9_\/-]+)(\.tex)*}?)|(%+\s*\\include\s*{?([A-Za-z0-9\/_-]+)(\.tex)*}?)|(\\input\s*{?([A-Za-z0-9_\/-]+)(\.tex)*}?)|(\\include\s*{?([A-Za-z0-9\/_-]+)(\.tex)*}?)'
pattern4 = r'/(\\bibliographystyle\s*{?([A-Za-z0-9_\/-]+)(\.tex)*}?)|(%+\s*\\bibliographystyle\s*{?([A-Za-z0-9_\/-]+)(\.tex)*}?)|(\\bibliography\s*{?([A-Za-z0-9_\/-]+)(\.tex)*}?)|(%+\s*\\bibliography\s*{?([A-Za-z0-9_\/-]+)(\.tex)*}?)'

#---------------------------------------------------
def check_bbl_exists(directory):
    # walk through the directory and check if there is a .bbl file
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.bbl'):
                return os.path.splitext(file)[0]
    return None


#-----------------------------------------------
def get_root_files(directory):
    root_files=[]
    # if len(os.listdir(directory))==1 and os.listdir(directory)[0].endswith('.tex'):
    #     return(root_files)
    name=check_bbl_exists(directory)
    if name:
        if name+'.tex' in os.listdir(directory):
            root_files.append(name+'.tex')
            return root_files
#---------------------------------------------------    
    
    for file in os.listdir(directory):
        if file.endswith('.tex'):
            with open(os.path.join(directory,file), 'rb') as f:
                try:
                    str=f.read()
                    try:
                        str = str.decode("utf-8")  # Ignore non-UTF-8 characters
                    except:
                        str = str.decode("latin-1", errors="ignore")

                    res=re.findall(pattern2,str)
                    if len(res) and (res[0] != "" or res[1] != "" or res[2] != ""):
                        root_files.append(file)
                except Exception as e:
                    print("%"*50,"\nError in file: ",file, ": ", directory, e)
    return(root_files)

def get_include_file_name_from_root_file(root_file):
    with open(root_file, 'rb') as f:
        try:
            str=f.read()
            try:
                str = str.decode("utf-8")  # Ignore non-UTF-8 characters
            except:
                str = str.decode("latin-1", errors="ignore")
            
            # res=re.findall(pattern3,str)
            include_files = re.findall(pattern3, str)
            file_names_to_be_included = []
            for include_file in include_files:
                if include_file[5] != "":
                    file_names_to_be_included.append(include_file[5])
                if include_file[7] != "":
                    file_names_to_be_included.append(include_file[7])
        except Exception as e:
            print(f'Errr in {root_file}: {e}')
            
    return list(set(file_names_to_be_included))
            
# Function to recursively merge input files
def merge_input(file_name1, file_name2, root_file_path):
    input_folder, root_file_name = os.path.dirname(root_file_path), os.path.basename(root_file_path)
    file_name = file_name1 if file_name1 != "" else file_name2
    if file_name == None or file_name == "":
        return ""
    print(f"Processing {file_name}")
    print(f'input_folder = {input_folder} file_name = {file_name} root_file_name = {root_file_name}')
    file_path = os.path.join(input_folder, file_name)
    # print(f"{file_name} == {root_file_name} or {file_name+'.bbl'} == {root_file_name}")
    try:
        if file_name == root_file_name or file_name+'.tex' == root_file_name:
            print("importing bibliography..................")
            tex_content = ""
            with open(file_path+".bbl", "rb") as tex_file:
                tex_content = tex_file.read()
        else:
            with open(file_path+".tex", "rb") as tex_file:
                tex_content = tex_file.read()
        try:
            tex_content = tex_content.decode("utf-8")
        except:
            tex_content = tex_content.decode("latin-1", errors="ignore")
        return tex_content
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return ""  # Return an empty string if the file is missing
    except Exception as e:
        print(f"Error in {file_path}: {e}")
        return ""
def merge_bibliography(file_name, root_file_path):
    if file_name  ==  "":
        return ""
    print(f"Processing {file_name}")
    # get the content from the .bbl file the same folder by walking through the directory
    input_folder, root_file_name = os.path.dirname(root_file_path), os.path.basename(root_file_path)
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.bbl'):
                with open(os.path.join(root,file), 'rb') as f:
                    try:
                        str=f.read()
                        try:
                            str = str.decode("utf-8")  # Ignore non-UTF-8 characters
                        except:
                            str = str.decode("latin-1", errors="ignore")
                        return str
                    except Exception as e:
                        print(f'Errr in {root_file_path}: {e}')
    print(f"File not found: {file_name}")
    return ""  # Return an empty string if the file is missing

def replace_import_with_file_data(root_file, output_folder_path):
    with open(os.path.join(root_file), 'rb') as f:
        # try:
            str=f.read()
            str = str.decode("utf-8", errors="ignore")  # Ignore non-UTF-8 characters
            # merged_content = re.sub(pattern3, lambda match: merge_input(match.group(6), match.group(8), os.path.dirname(root_file)), str)
            # merge the tex file with input or include command
            merged_content = re.sub(pattern3, lambda match: merge_input(match.group(8), match.group(11), root_file), str)
            # merge the bbl file with bibliography command
            merged_content = re.sub(pattern4, lambda match: merge_bibliography(match.group(7), root_file), merged_content)

            # Write the merged content to the output folder with subfolder name
            output_filename = f"{os.path.basename(os.path.dirname(root_file))}.tex"
            print(f"Writing {output_filename}")
            print(f"Writing {output_folder_path}")
            output_path = os.path.join(output_folder_path, output_filename)
            with open(output_path, "w", encoding="utf-8") as output_tex:
                output_tex.write(merged_content)
        # except Exception as e:
        #     print(f'Errr in {root_file}: {e}')            
            

###

In [4]:
# read the csv file
import csv
directories = []
with open('2302_no_main_tex.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        directories.append(row[0])

len(directories), directories[0]

(496, '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00678')

In [5]:
import os
n = "file.tex.mp4.mp3"
# get the name of the file without extension
os.path.splitext(n)[0]

'file.tex.mp4'

In [6]:


os.listdir(directories[0])

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00678'

In [7]:
# for i in range(100,115):
#     print(get_root_files(directories[i]))
#old version : where we are not checking for bbl file

# ['final_manuscript.tex']
# ['ms.tex']
# ['proposal.tex', 'midterm-report.tex', 'paper.tex']
# ['robchat_without_prompt.tex']
# ['elsarticle-template-num.tex']
# ['syskuramotovar.tex', 'syskuramotovar-supplement.tex']
# ['Abundance_ratio.tex', 'aassymbols.tex', 'natbib.tex', 'natnotes.tex']
# ['AMP_init.tex']
# ['EM-waveguidemodes.tex']
# ['ckFermionBootstrap.tex']
# ['ex_supplement.tex', 'docsiamart.tex', 'Decomposed_neural_networks_for_image_recognition.tex']
# ['Summary_of_Revisions.tex', 'SIGMOD.tex']
# ['iclr_2023.tex']
# ['SWMVI_ICLR2023.tex']
# ['uai2022.tex']

for i in range(100,115):
    print(get_root_files(directories[i]))

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04432'

In [8]:
# for dir in directories:
#     # get the directory that contains a subdirectory in it.
#     for subdir in os.listdir(dir):
#         if os.path.isdir(os.path.join(dir,subdir)):
#             print(os.path.join(dir,subdir))
#             input()


    

In [8]:
file = "/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09369/main.tex"
os.path.basename(os.path.dirname(file))

'2302.09369'

In [9]:
os.path.dirname(file)

'/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09369'

In [10]:
# get the file name only from file variable
os.path.basename(file)

'main.tex'

### Testing in the test enviroment

- replace the parent_dir path

In [11]:
# # get the parent directoy path from directories[0]
# parent_dir = os.path.dirname(directories[0])
import re
import os
parent_dir = "/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09369/"
parent_dir = "/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369"

In [12]:
root_file = get_root_files(parent_dir)
root_file

['iclr2023_conference.tex']

In [13]:
os.listdir(parent_dir)

['2302.09369_old2.tex',
 '2302.09369.tex',
 'math_commands.tex',
 'figs',
 '2302.09369_old.tex',
 'iclr2023_conference.bst',
 'sections',
 'fancyhdr.sty',
 'iclr2023_conference.tex',
 'iclr2023_conference.sty',
 'iclr2023_conference.bbl',
 'iclr2023_conference.bib',
 'natbib.sty']

In [15]:

strs = '''
\input{sections/_a_Appendix}
\input {sections/_a_Appendix}
\input sections/_a_Appendix
\include{sections/_a_Appendix}
\include {sections/_a_Appendix}
\include sections/_a_Appendix

\input{sections/_a_Appendix.tex}
\input {sections/_a_Appendix.tex}
\input sections/_a_Appendix.tex
\include{sections/_a_Appendix.tex}
\include {sections/_a_Appendix.tex}
\include sections/_a_Appendix.tex

\input{sections/_a_Appendix.bbl}
\input {sections/_a_Appendix.bbl}
\input sections/_a_Appendix.bbl
\include{sections/_a_Appendix.bbl}
\include {sections/_a_Appendix.bbl}
\include sections/_a_Appendix.bbl

\input{a.tex}
\input {a.tex}
\input a.tex 
\input{a}
\input {a}
\input a

\include{a.tex}
\include {a.tex}
\include a.tex
\include{a}
\include {a}
\include a

% ritesh \input{sections/_a_Appendix}
% \include sections/_a_Appendix
%\input{sections/_a_Appendix}
%\include sections/_a_Appendix
%  \input{sections/_a_Appendix}
%  \include sections/_a_Appendix
%% \input{sections/_a_Appendix}
%% \include sections/_a_Appendix
'''

str2 = '\input{sections/_a_Appendix}'

In [16]:
patternx = r'(%+\s*\\input\s*{?([A-Za-z0-9_\/-]+))|(%+\s*\\include\s*{?([A-Za-z0-9\/_-]+))|(\\input\s*{?([A-Za-z0-9_\/-]+))|(\\include\s*{?([A-Za-z0-9\/_-]+))'

include_files = re.findall(patternx, strs)
print(include_files)
file_names_to_be_included = []
for include_file in include_files:
    if include_file[5] != "":
        file_names_to_be_included.append(include_file[5])
    if include_file[7] != "":
        file_names_to_be_included.append(include_file[7])
set(file_names_to_be_included)

[('', '', '', '', '\\input{sections/_a_Appendix', 'sections/_a_Appendix', '', ''), ('', '', '', '', '\\input {sections/_a_Appendix', 'sections/_a_Appendix', '', ''), ('', '', '', '', '\\input sections/_a_Appendix', 'sections/_a_Appendix', '', ''), ('', '', '', '', '', '', '\\include{sections/_a_Appendix', 'sections/_a_Appendix'), ('', '', '', '', '', '', '\\include {sections/_a_Appendix', 'sections/_a_Appendix'), ('', '', '', '', '', '', '\\include sections/_a_Appendix', 'sections/_a_Appendix'), ('', '', '', '', '\\input{sections/_a_Appendix', 'sections/_a_Appendix', '', ''), ('', '', '', '', '\\input {sections/_a_Appendix', 'sections/_a_Appendix', '', ''), ('', '', '', '', '\\input sections/_a_Appendix', 'sections/_a_Appendix', '', ''), ('', '', '', '', '', '', '\\include{sections/_a_Appendix', 'sections/_a_Appendix'), ('', '', '', '', '', '', '\\include {sections/_a_Appendix', 'sections/_a_Appendix'), ('', '', '', '', '', '', '\\include sections/_a_Appendix', 'sections/_a_Appendix'),

{'a', 'sections/_a_Appendix'}

In [17]:
parent_dir, root_file[0]

('/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369',
 'iclr2023_conference.tex')

In [18]:
get_include_file_name_from_root_file(os.path.join(parent_dir, root_file[0]))

['sections/_d_Discuss',
 'sections/_s6_conclusion',
 'sections/_c_Results',
 'sections/_s5_experiments',
 'sections/_s2_related_work',
 'math_commands',
 'sections/_b_Bayes_approx',
 'sections/_s4_reliable',
 'sections/_a_Appendix',
 'sections/_s3_method',
 'iclr2023_conference',
 'sections/_s1_introduction']

In [19]:
os.path.join(parent_dir, root_file[0])

'/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369/iclr2023_conference.tex'

##

In [20]:
output_folder_path = parent_dir
replace_import_with_file_data(os.path.join(parent_dir, root_file[0]), output_folder_path)

Processing math_commands
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369 file_name = math_commands root_file_name = iclr2023_conference.tex
Processing sections/_s1_introduction
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369 file_name = sections/_s1_introduction root_file_name = iclr2023_conference.tex
Processing sections/_s2_related_work
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369 file_name = sections/_s2_related_work root_file_name = iclr2023_conference.tex
Processing sections/_s3_method
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369 file_name = sections/_s3_method root_file_name = iclr2023_conference.tex
Processing sections/_s4_reliable
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369 file_name = sections/_s4_reliable root_file_name = iclr2023_conference.tex
Processing sections/_s5_experiments
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.09369 file_name = se

### Testing the patten4 merging

In [21]:
import re
import os
parent_dir = "/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.00217"
print(f"All the files in the directory: {os.listdir(parent_dir)}")
root_file = get_root_files(parent_dir)
print(f'Root file: {root_file}')
output_folder_path = parent_dir
replace_import_with_file_data(os.path.join(parent_dir, root_file[0]), output_folder_path)

All the files in the directory: ['main.bbl', 'main.tex', '2302.00217.tex', 'figures', 'ref.bib']
Root file: ['main.tex']
Processing \bibliographystyle
Processing \bibliography{ref}
Writing 2302.00217.tex
Writing /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.00217


# final py file

In [22]:
year_directory = '/mnt/NFS/patidarritesh/PDF_2_TEX/2023'

# we will have one folder final_extracted 
# which contains month wise folders
# create final_merged folder with same subfolders as final_extracted

# create a folder final_merged
if not os.path.exists(os.path.join(year_directory, 'final_merged')):
    os.mkdir(os.path.join(year_directory, 'final_merged'))

# create month wise folders in final_merged
for month_folder in os.listdir(os.path.join(year_directory, 'final_extracted')):
    if not os.path.exists(os.path.join(year_directory, 'final_merged', month_folder)):
        os.mkdir(os.path.join(year_directory, 'final_merged', month_folder))

# Each month folder will have a list of directories in it which are the extracted zip of each paper

for month_folder in os.listdir(os.path.join(year_directory, 'final_extracted')):
    for paper_folder in os.listdir(os.path.join(year_directory, 'final_extracted', month_folder)):
        
        # process the paper folder and write the merged file in the respective month folder
        # parent_dir = "/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.00217"
        parent_dir = os.path.join(year_directory, 'final_extracted', month_folder, paper_folder)
        output_folder_path = os.path.join(year_directory, 'final_merged', month_folder)
        # print(f"All the files in the directory: {os.listdir(parent_dir)}")
        count_tex = 0
        count_bbl = 0
        bbl_file = ''

        for root, dir, files in os.walk(parent_dir):
            for file in files:
                if file.endswith('.tex') or file.endswith('.TEX'):
                    count_tex+=1
                if file.endswith('.bbl') or file.endswith('.BBL'):
                    count_bbl+=1
                    bbl_file = file
        
        # print(count_tex, count_bbl)
        # print(paper_folder)
        if(count_tex>1):
            try:
                root_file = get_root_files(parent_dir)
                print(f'Paper {paper_folder} Root file: {root_file}')
                replace_import_with_file_data(os.path.join(parent_dir, root_file[0]), output_folder_path)
            except:
                print(f'Error in {parent_dir}')
        elif(count_tex == 1):
            for root, dir, files in os.walk(parent_dir):
                for file in files:
                    try:
                        if file.endswith('.tex') or file.endswith('.TEX'):
                            if count_bbl == 0:
                                # find the path of tex file
                                # copy the tex file to the output folder renaming as the paper folder name
                                os.rename(os.path.join(parent_dir, file), os.path.join(parent_dir, paper_folder+'.tex'))
                                print(f"Successfully renamed {file} to {paper_folder+'.tex'}")

                                file_path = os.path.join(parent_dir, paper_folder+'.tex')

                                subprocess.run(["cp", file_path, output_folder_path], check=True)
                                print(f"Successfully copied {file} to {output_folder_path}")
                            else:
                                replace_import_with_file_data(os.path.join(parent_dir, file), output_folder_path)
                            break
                    except:
                        print(f'error in {parent_dir}')
                        # lst.append(parent_dir)

Processing None
Processing None
Processing \bibliography{References}
Writing 2302.05057.tex
Writing /mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302
Processing \bibliographystyle
Processing \bibliography{BibFile
Writing 2302.10370.tex
Writing /mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302
Processing \bibliographystyle
Processing \bibliography{uai2023-template}
Writing 2302.12456.tex
Writing /mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302
Processing \bibliography{dag
Processing \bibliographystyle
Writing 2302.14834.tex
Writing /mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302
Successfully renamed PtYIG-IL-gating-8.tex to 2302.09505.tex
Successfully copied PtYIG-IL-gating-8.tex to /mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302
Processing \bibliographystyle
Processing \bibliography{lizcites}
Writing 2302.06716.tex
Writing /mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302
Paper 2302.06726 Root file: ['main.tex']
Processing macros
input_folder = 

In [28]:
parent_dir = '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.14402'
get_root_files(parent_dir)

['PaperCombineArxiv.tex']

In [31]:
len(os.listdir('/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302'))

4104

In [33]:
parent_dir = '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.12995'
output_folder_path = '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.12995'

root_file = get_root_files(parent_dir)
print(root_file)
print(f'Paper {paper_folder} Root file: {root_file}')
replace_import_with_file_data(os.path.join(parent_dir, root_file[0]), output_folder_path)

['PaperForReview.tex']
Paper 2302.02437 Root file: ['PaperForReview.tex']
Processing content/intro_new
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.12995 file_name = content/intro_new root_file_name = PaperForReview.tex
Processing content/related_work
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.12995 file_name = content/related_work root_file_name = PaperForReview.tex
Processing content/methodology
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.12995 file_name = content/methodology root_file_name = PaperForReview.tex
Processing content/experiments
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.12995 file_name = content/experiments root_file_name = PaperForReview.tex
Processing content/conclusion
input_folder = /mnt/NFS/patidarritesh/PDF_2_TEX/2023/Test/2302.12995 file_name = content/conclusion root_file_name = PaperForReview.tex
Processing \bibliographystyle
Processing \bibliography{egbib}
Writing 2302.12995.tex
Writi

In [36]:
directories =['/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00678', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02031', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01317', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12977', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08580', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06497', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00660', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13807', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14428', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03725', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02916', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03807', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08511', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09369', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13837', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12995', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11979', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02074', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02016', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09042', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07205', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00832', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04237', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04233', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07224', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02908', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03773', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14380', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00785', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04496', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12193', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05798', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08570', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06476', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02386', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00353', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11821', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11898', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14417', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05892', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07529', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02910', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03915', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02899', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13820', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02419', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07637', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12100', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13935', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04393', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11656', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06594', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09056', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06302', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06289', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09123', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11855', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07179', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12898', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06455', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13382', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14399', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11704', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12172', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07555', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12139', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01203', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07505', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11861', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12958', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14372', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14413', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07176', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11467', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07994', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02533', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02341', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00219', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00787', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07942', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13865', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04223', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06241', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11505', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04464', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02477', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09010', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00760', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02909', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08575', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09048', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06514', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03762', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03926', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07950', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06606', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08519', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00620', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13800', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00842', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04432', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00293', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03751', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12095', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02924', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06326', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12970', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03682', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11994', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14861', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06564', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11974', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13812', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09029', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12196', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04141', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11874', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06353', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02897', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13289', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07204', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00594', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00849', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00255', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02285', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08469', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02933', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03462', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00690', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09106', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06412', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02893', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03813', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02876', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00675', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09137', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14362', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08502', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02882', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00758', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02460', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06378', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13405', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14426', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04434', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02915', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14422', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04382', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06576', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07164', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14367', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13370', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03933', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13909', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07648', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02110', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13019', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06637', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12884', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03825', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09273', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07563', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00773', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00588', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06440', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11546', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05794', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02497', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02859', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02310', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02483', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05783', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03869', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03784', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06255', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04196', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13001', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06423', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13285', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02040', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11530', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08501', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07628', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02895', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02420', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02535', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09018', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13325', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01265', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09376', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12006', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03860', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09083', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03474', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07940', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06478', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09384', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13319', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00626', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04376', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12068', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08643', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03880', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13007', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01244', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13806', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03844', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02307', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13782', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13002', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06607', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00227', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12049', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01269', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03495', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11524', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03868', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02540', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12929', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07919', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09359', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04388', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07549', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08527', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07599', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04481', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02328', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00341', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00319', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00365', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00608', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13940', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04375', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06356', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08617', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00374', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04441', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11850', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03775', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04165', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14457', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13425', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12066', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01209', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09344', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02372', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03664', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11565', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13395', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00621', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03679', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00242', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13834', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11893', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01252', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04179', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02392', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08475', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07194', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00268', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02488', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11988', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12086', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03499', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11431', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11603', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12895', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12188', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02345', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08555', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06491', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04207', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02065', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08466', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02346', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04226', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07250', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02254', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11654', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09386', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04156', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08479', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08646', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14395', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11619', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07973', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11623', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11844', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02299', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00263', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03802', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03429', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07983', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00813', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04488', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03826', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06632', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07511', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02537', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02049', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04363', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11665', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09087', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11683', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13312', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00814', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02429', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02466', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00228', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07217', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07937', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04181', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00316', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09370', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11556', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07570', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03628', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09086', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11831', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05800', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03859', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02044', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05811', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06528', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07576', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03408', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04453', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03432', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06323', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03744', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08532', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12948', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00624', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02017', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12117', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03855', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13005', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14365', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13880', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05882', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07627', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11651', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01259', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07155', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07630', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06502', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03650', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11466', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06600', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06611', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12058', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13373', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12120', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07167', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09324', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04192', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03789', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13394', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06553', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09261', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12031', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11510', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07922', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13824', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14455', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01226', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02070', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04459', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00282', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03921', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00771', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04175', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07634', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00808', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00350', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08647', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13344', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03502', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07545', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06556', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03662', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09053', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11816', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08520', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02023', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05894', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02522', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02298', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07574', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06579', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12126', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13369', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04369', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04511', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02006', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02870', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02879', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11529', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07536', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11575', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.14383', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03777', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08531', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00801', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02887', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00377', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12187', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08600', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00802', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04445', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13024', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13904', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05889', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01256', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00581', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11826', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11569', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07255', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03715', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02294', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13915', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03899', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07182', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13844', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04479', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04222', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00244', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02080', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04463', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03791', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02325', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06546', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11902', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11891', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06385', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12998', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13843', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05812', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02888', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04452', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09354', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11621', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06527', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04229', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12042', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11537', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02013', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07534', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01312', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11873', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07253', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00693', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03829', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02863', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13436', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03845', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07223', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11825', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12045', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04234', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09103', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08672', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11609', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13899', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06266', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12951', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11817', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05862', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00601', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00782', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.05813', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02064', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00286', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00705', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.07659', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.06263', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12194', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11703', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.04188', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.01263', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.08692', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02482', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02407', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12944', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09263', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03396', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.09276', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02314', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03811', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11622', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.11864', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.13365', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.02900', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.12988', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.00666', '/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302/2302.03694']


In [41]:
# There are paths in directories list, check waht path in directories list is not present in "/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302"
lst = []
count = 0
for file in directories:
    file = (os.path.basename(file))+'.tex'
    if  file in os.listdir("/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302"):
        
        count += 1
    else:
        lst.append(file)
        
print(count)
print(lst)

495
['2302.02466.tex']


In [23]:
len(os.listdir('/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_extracted/2302'))

4091

In [24]:
len(os.listdir('/mnt/NFS/patidarritesh/PDF_2_TEX/2023/target_src/2302'))

4469

In [25]:
len(os.listdir('/mnt/NFS/patidarritesh/PDF_2_TEX/2023/final_merged/2302'))

4083

In [17]:
strs = "fgfh\n\n\n\n\nhsfsfnfs"
with open('test.txt', 'w') as f:
    f.write(strs)

In [16]:
import re

strs = re.sub('\n+','\n',strs)
# print(strs.replace('\n\n','\n'))
print(strs)

fgfh
hsfsfnfs


In [18]:
with open('test.txt', 'r+') as f:
    strs = f.read()
    print(strs)
    f.seek(0)
    f.write(re.sub('\n+','\n',strs))
    


fgfh




hsfsfnfs
